In [6]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

batch_size = 32
block_size = 512


def get_batch(data):
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [7]:
data = torch.randn((1000))
data.size()

torch.Size([1000])

In [8]:
x, y = get_batch(data)
x.size(), y.size()

tensor([298,  36, 485, 398, 478, 415, 213, 414,  75, 174, 174, 215, 292, 463,
        228, 216, 394, 471, 265, 468, 387,  70, 167, 389,   1, 185, 457, 346,
        203,  20, 316, 144])


(torch.Size([32, 512]), torch.Size([32, 512]))

In [10]:
x

tensor([[ 0.6622, -0.1713,  0.0137,  ...,  1.8254, -1.7514,  0.5758],
        [ 0.4835,  0.2532,  0.1748,  ...,  0.3878,  1.3287,  1.0314],
        [ 0.8357, -0.1912, -0.4461,  ...,  0.0972, -0.1021, -0.7441],
        ...,
        [ 0.4724,  0.1179, -0.2373,  ..., -0.1339,  0.4963, -0.8573],
        [-0.4516, -0.6836,  1.6792,  ..., -0.9584, -1.3133, -1.8018],
        [-1.5091,  1.3658,  0.0984,  ..., -0.2807,  0.9801,  0.7553]])

In [1]:
import torch.nn as nn
import torch
from torch.nn import functional as F
from Embedding.CausalConv1D import CausalConv1d
from torch import Tensor
from torch.nn import Conv1d 
 

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [3]:
class SoftClip(nn.Module):
    def __init__(self, bound: float = 1.0):
        super().__init__()

        self.bound = bound

    def forward(self, x: Tensor) -> Tensor:
        return x / (1 + abs(x / self.bound))

In [26]:

layers = nn.Sequential(
            SoftClip(100.0),
            CausalConv1d(in_channels=2, out_channels=32, dilation=1, kernel_size=32, A=True, bias = True, stride = 2), #
            nn.LeakyReLU(), 
            CausalConv1d(in_channels=32, out_channels=32, dilation=2, kernel_size=64, A=False, bias = True, stride = 2), 
            nn.LeakyReLU(), 
            CausalConv1d(in_channels=32, out_channels=32, dilation=4, kernel_size=128, A=False, bias = True, stride = 2), 
            nn.LeakyReLU(), 
            CausalConv1d(in_channels=32, out_channels=32, dilation=8, kernel_size=32, A=False, bias = True,  stride = 2), 
            nn.LeakyReLU(), 
            # CausalConv1d(in_channels=32, out_channels=32, dilation=16, kernel_size=32, A=False, bias = True), 
            # nn.LeakyReLU(), 
            # CausalConv1d(in_channels=32, out_channels=64, dilation=32, kernel_size=32, A=False, bias = True), 
            # nn.LeakyReLU(), 
            
            # Conv1d(in_channels=64, out_channels=128, kernel_size=64, stride= 32), 
            # nn.LeakyReLU(),
            # Conv1d(in_channels=128, out_channels=64, kernel_size=64, stride= 32), 
            # nn.LeakyReLU(),
             )


In [27]:
x = torch.randn((16, 2, 6144))
# x = torch.randn((16, 2, 8))

In [28]:
x = layers(x)
print(x.size())
f = nn.Flatten()
f(x).size()

torch.Size([16, 32, 116])


torch.Size([16, 3712])

In [76]:
x.size()

torch.Size([16, 64, 4])